In [55]:
import pandas as pd
import re #정규표현식 전처리
import matplotlib.pyplot as plt #시각화
import numpy as np
import datetime

from matplotlib import font_manager, rc #한글폰트
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [56]:
vod = pd.read_csv('./LG_data/01. 전처리/vod_전처리_19.csv', encoding='cp949')

# 전처리

In [57]:
# 불필요한 컬럼제거 
vod.drop(['Unnamed: 0'], axis=1, inplace=True)

# 로그데이터 datetime으로 만들기
vod['strt_dt'].unique()
date = pd.to_datetime(vod.strt_dt, format = "%Y%m%d%H%M%S")
vod['datetime'] = date
vod['day'] = vod['datetime'].apply(lambda x : x.day)
vod['day'].unique() #결측치 없음
vod['day'].fillna(-1, inplace= False) 
vod['day'] = vod['day'].astype(int)
vod['hour'] = vod['datetime'].apply(lambda x : x.hour)
vod['hour'].unique() #결측치 없음
vod['hour'].fillna(-1, inplace= False)
vod['hour'] = vod['hour'].astype(int)
vod.reset_index(drop= False, inplace= False)
# 00:00 형식의 datetiem -> n분으로 변경 1:43 = 103분
vod['disp_m'] = vod['disp_rtm'].dropna().apply(lambda x: int(str(x).split(':')[0]) * 60 + int(str(x).split(':')[1]) if ':' in str(x) else float('nan'))
vod['disp_m'].isna().sum()
# 실수형 -> 정수형 형변환
vod['disp_m'] = vod['disp_m'].fillna(0).astype(int)

### 4. contents 페이지에 머무른 시간이 실제 시청 시간에 영향을 미치는지

In [58]:
#vod가 예고 회차 포함, vod_1이 제목 전처리
vod_4 = vod[['subsr', 'asset_nm', 'datetime', 'use_tms']] #17424
vod_4.dropna(subset = ['datetime'], inplace=True)
vod_4

C:\Users\USER\AppData\Local\Temp\ipykernel_7780\1854554472.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_4.dropna(subset = ['datetime'], inplace=True)


,subsr,asset_nm,datetime,use_tms
0,66875000,장군재상 10회(A),2024-01-23 08:51:02,2472
1,66705000,수호지의 무송 04회,2024-01-16 20:37:41,1891
2,66705000,수호지의 무송 04회,2024-01-17 19:31:53,632
3,66437000,콩순이 율동교실 4기 01회,2024-01-13 18:42:21,60
4,66841000,알사탕,2024-01-13 07:43:50,480
...,...,...,...,...
17419,64399000,(자막)주술회전 2기: 시부야 사변 16회(N),2024-03-02 23:17:15,1276
17420,63086000,괴물(2023)(예고),2024-03-10 19:05:08,60
17421,63017000,만난지3분만에합체 저랑섹스한번하실래요(예고),2024-03-30 10:39:45,60
17422,66056000,만난지3분만에합체 저랑섹스한번하실래요(예고),2024-03-30 17:16:45,60


In [59]:
contents = pd.read_csv('./LG_data/01. 전처리/contents_전처리_19.csv', encoding= 'cp949') #73247
contents.drop(['Unnamed: 0'], axis=1, inplace=True)

date = pd.to_datetime(contents.log_dt, format = "%Y%m%d%H%M%S")
contents['datetime'] = date 
contents.reset_index(drop=True, inplace=True)
# 00:00 형식의 datetiem -> n분으로 변경
contents['disp_m'] = contents['disp_rtm'].dropna().apply(lambda x: int(str(x).split(':')[0]) * 60 + int(str(x).split(':')[1]) if ':' in str(x) else float('nan'))
contents['disp_m']

0          NaN
1          NaN
2        116.0
3        112.0
4        112.0
         ...  
73242     45.0
73243     30.0
73244     30.0
73245     46.0
73246     46.0
Name: disp_m, Length: 73247, dtype: float64

In [60]:
contents_4 = contents[['subsr', 'super_asset_nm', 'datetime']]
contents_4.columns = ['subsr', 'asset_nm', 'datetime'] #73247
contents_4.dropna(subset = ['datetime'], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_7780\887407042.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_4.dropna(subset = ['datetime'], inplace=True)


In [61]:
preview_list = ['0회', '00회', '000회', '(예고)', '(사전구매)', '(커밍순)', '(예고편)', '(예약구매)', '비하인드', '(예고편 1)',
                ] 
preview_keyword = '|'.join(preview_list)
vod_4['preview'] = vod_4['asset_nm'].str.contains(preview_keyword, na=False)
vod_4

contents_4['preview'] = contents_4['asset_nm'].str.contains(preview_keyword, na=False)

vod_5 = vod_4[vod_4['preview'] == False]
contents_5 = contents_4[contents_4['preview'] == False]

vod_5
contents_5['asset_nm']

C:\Users\USER\AppData\Local\Temp\ipykernel_7780\3997462057.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vod_4['preview'] = vod_4['asset_nm'].str.contains(preview_keyword, na=False)
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\3997462057.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_4['preview'] = vod_4['asset_nm'].str.contains(preview_keyword, na=False)
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\3997462057.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contents_4['preview'] = contents_4['asset_nm'].str.contains(preview_keyword, na=False)
C:\U

0                           유혹2
1                          2차노출
2                           마들렌
3                       1번가의 기적
4                       1번가의 기적
                  ...          
73242    매화홍도 : 미스터 앤 미세스 첸 24회
73243                눈떠보니 라떼 3회
73244                눈떠보니 라떼 3회
73245              내 이야기는 길어 1회
73246              내 이야기는 길어 1회
Name: asset_nm, Length: 69433, dtype: object

#### 회차 제거

vod/contents의 asset_nm의 ()를 포함한 문자열 제거

In [62]:
# 프로그램 이름 정규화 함수
# 회차 정보는 남기기
def normalize_program_name(name):
    # 대괄호 제거 ([설 특집 영화]비공식작전 -> 비공식작전)
    name = re.sub(r'\([^)]*\)', '', name).strip()
    name = re.sub(r'\[.*?\]', '', name).strip()
    name = re.sub(r'\d+회', '', name)

    return name

In [63]:
# 프로그램 이름 정규화 적용 - 회차 제거
vod_5['asset_nm'] = vod_5['asset_nm'].apply(normalize_program_name)

contents_5['asset_nm'] = contents_5['asset_nm'].apply(normalize_program_name)

C:\Users\USER\AppData\Local\Temp\ipykernel_7780\2829468.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_5['asset_nm'] = vod_5['asset_nm'].apply(normalize_program_name)
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\2829468.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_5['asset_nm'] = contents_5['asset_nm'].apply(normalize_program_name)


In [64]:
vod_5['strt_date'] = vod_5['datetime'].dt.date
contents_5['strt_date'] = contents_5['datetime'].dt.date


vod_5['time'] = vod_5['datetime'].dt.time
contents_5['time'] = contents_5['datetime'].dt.time

C:\Users\USER\AppData\Local\Temp\ipykernel_7780\1923516231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_5['strt_date'] = vod_5['datetime'].dt.date
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\1923516231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_5['strt_date'] = contents_5['datetime'].dt.date
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\1923516231.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [65]:
first_start_times = vod_5.groupby(['subsr', 'asset_nm']).agg({'use_tms': 'sum', 'strt_date' : 'min', 'datetime' : 'min'}).reset_index()
first_access_times = contents_5.groupby(['subsr', 'asset_nm']).agg({'strt_date' : 'min', 'datetime' : 'min'}).reset_index()

# print(first_start_times) #4066
# print(first_access_times) #21393

In [66]:
content_based_merged = pd.merge(first_access_times, first_start_times, on=['subsr', 'asset_nm'], how='left', suffixes=('_access', '_start'))

In [67]:
content_based_merged['time_difference'] = content_based_merged['datetime_start'] - content_based_merged['datetime_access']
content_based_merged #21393

#5초 미만 제거
content_based_merged = content_based_merged[(content_based_merged['time_difference'].isna())| (content_based_merged['time_difference'] >= pd.Timedelta(seconds=5))]
content_based_merged #20629

,subsr,asset_nm,strt_date_access,datetime_access,use_tms,strt_date_start,datetime_start,time_difference
0,59879000,간신,2024-01-19,2024-01-19 05:26:23,NaN,NaN,NaT,NaT
2,59880000,도그데이즈,2024-03-09,2024-03-09 09:22:41,NaN,NaN,NaT,NaT
3,59882000,한글용사 아이야 시즌3,2024-03-24,2024-03-24 08:32:25,NaN,NaN,NaT,NaT
4,59883000,마크맨,2024-02-15,2024-02-15 22:53:40,NaN,NaN,NaT,NaT
5,59883000,서울의 봄,2024-02-15,2024-02-15 22:23:13,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...
21388,67400000,아내 대신 채워준 야한 장모님,2024-03-31,2024-03-31 22:32:16,NaN,NaN,NaT,NaT
21389,67400000,유산으로 남겨준 새엄마,2024-03-31,2024-03-31 22:31:52,NaN,NaN,NaT,NaT
21390,67400000,조건섹스 : 라스트 원나잇,2024-03-31,2024-03-31 22:49:22,NaN,NaN,NaT,NaT
21391,67400000,편의점 털다 아랫도리 털린 유부녀,2024-03-31,2024-03-31 22:31:58,NaN,NaN,NaT,NaT


In [68]:
#기본적으로 inner join - 왼쪽 데이터 프레임을 기준으로 병합하고 오른쪽프레임에 해당하는 행이 없는 경우 행을 삭제
#datetime_access가 없는 경우에 drop함
vod_content_merged_num = pd.merge(first_start_times, first_access_times, on=['subsr', 'asset_nm'], suffixes=('_start', '_access'))
vod_content_merged_num #3506

,subsr,asset_nm,use_tms,strt_date_start,datetime_start,strt_date_access,datetime_access
0,59879000,남편은 이렇게 못해줘,940,2024-01-09,2024-01-09 20:28:46,2024-01-09,2024-01-09 20:28:44
1,59883000,표적,1310,2024-02-15,2024-02-15 22:31:28,2024-02-15,2024-02-15 22:31:11
2,59899000,0시,160,2024-03-01,2024-03-01 21:27:35,2024-03-01,2024-03-01 21:27:32
3,59899000,겨울부인,119,2024-03-01,2024-03-01 21:39:13,2024-03-01,2024-03-01 21:39:10
4,59899000,경마장가는길,456,2024-03-01,2024-03-01 21:41:26,2024-03-01,2024-03-01 21:41:24
...,...,...,...,...,...,...,...
3501,67356000,블라인드,3890,2024-03-18,2024-03-18 19:36:56,2024-03-18,2024-03-18 19:36:33
3502,67356000,색에 놀다,1966,2024-02-25,2024-02-25 19:24:02,2024-02-25,2024-02-25 19:23:45
3503,67356000,섹스 앤 머니,5271,2024-02-26,2024-02-26 08:39:23,2024-02-26,2024-02-26 08:39:10
3504,67356000,아내의 속사정:음란한 거래,3817,2024-03-07,2024-03-07 05:57:48,2024-03-07,2024-03-07 05:52:18


In [69]:
vod_content_prior = vod_content_merged_num[vod_content_merged_num['strt_date_start'] > vod_content_merged_num['strt_date_access']]
vod_content_prior #725

,subsr,asset_nm,use_tms,strt_date_start,datetime_start,strt_date_access,datetime_access
8,59900000,고려거란전쟁,4332,2024-03-05,2024-03-05 11:20:54,2024-01-03,2024-01-03 14:36:08
30,59937000,색에 놀다,133,2024-02-21,2024-02-21 01:52:01,2024-02-03,2024-02-03 23:39:10
33,60020000,나폴레옹,527,2024-01-21,2024-01-21 17:20:33,2024-01-14,2024-01-14 12:52:06
34,60050000,97분,5295,2024-03-05,2024-03-05 01:45:01,2024-01-31,2024-01-31 23:42:15
35,60050000,도그맨,6587,2024-03-02,2024-03-02 16:40:42,2024-02-25,2024-02-25 00:04:39
...,...,...,...,...,...,...,...
3476,67303000,인간극장,7813,2024-02-17,2024-02-17 07:12:32,2024-02-16,2024-02-16 08:23:47
3482,67303000,호텔 델루나,7741,2024-03-17,2024-03-17 12:53:40,2024-02-27,2024-02-27 18:44:42
3488,67334000,노량: 죽음의바다,5951,2024-02-06,2024-02-06 13:46:00,2024-02-05,2024-02-05 14:07:33
3495,67334000,트롤: 밴드 투게더,240,2024-02-27,2024-02-27 15:49:00,2024-02-21,2024-02-21 13:57:25


In [70]:
vod_content_under_5 = vod_content_prior[vod_content_prior['use_tms'] < 300]
vod_content_under_5 #115

,subsr,asset_nm,use_tms,strt_date_start,datetime_start,strt_date_access,datetime_access
30,59937000,색에 놀다,133,2024-02-21,2024-02-21 01:52:01,2024-02-03,2024-02-03 23:39:10
111,60320000,1947 보스톤,114,2024-03-15,2024-03-15 23:15:17,2024-02-25,2024-02-25 23:03:25
127,60326000,상국부,242,2024-03-12,2024-03-12 03:25:41,2024-01-11,2024-01-11 22:47:13
163,60326000,청심협녀,160,2024-03-04,2024-03-04 07:44:03,2024-01-14,2024-01-14 03:11:28
199,60495000,라이브 플래쉬,299,2024-03-05,2024-03-05 14:28:56,2024-01-08,2024-01-08 14:18:03
...,...,...,...,...,...,...,...
3260,66705000,본 투 플라이,213,2024-02-16,2024-02-16 23:32:17,2024-01-24,2024-01-24 20:16:16
3281,66705000,왜구 토벌대,148,2024-02-21,2024-02-21 16:34:07,2024-01-18,2024-01-18 19:51:53
3287,66705000,일촌상사: 나의 소녀,162,2024-02-11,2024-02-11 14:12:54,2024-01-20,2024-01-20 17:29:01
3495,67334000,트롤: 밴드 투게더,240,2024-02-27,2024-02-27 15:49:00,2024-02-21,2024-02-21 13:57:25


##### 회차 제거한 경우

In [71]:
print(len(vod_content_prior) / len(content_based_merged) * 100)
print(len(vod_content_prior) / len(first_start_times) * 100)

3.5144699209850208
17.830791933103786


In [72]:
print(len(vod_content_under_5)/len(vod_content_prior))

0.15862068965517243


###### 상세페이지 접속의 vod 시청 전환율)  VOD 전환 / 상세페이지 = 725 / 20629 = 3.514% 

###### vod 시청 기록 중 상세페이지에서 넘어온 비율) 725 / 4066 = 17.83%

#### 회차 제거 없이

In [73]:
vod_5 = vod_4[vod_4['preview'] == False]
contents_5 = contents_4[contents_4['preview'] == False]

vod_5
contents_5['asset_nm']

0                           유혹2
1                          2차노출
2                           마들렌
3                       1번가의 기적
4                       1번가의 기적
                  ...          
73242    매화홍도 : 미스터 앤 미세스 첸 24회
73243                눈떠보니 라떼 3회
73244                눈떠보니 라떼 3회
73245              내 이야기는 길어 1회
73246              내 이야기는 길어 1회
Name: asset_nm, Length: 69433, dtype: object

In [74]:
# 프로그램 이름 정규화 함수
# 회차 정보는 남기기
def normalize_program_name_remain(name):
    # 대괄호 제거 ([설 특집 영화]비공식작전 -> 비공식작전)
    name = re.sub(r'\([^)]*\)', '', name).strip()
    name = re.sub(r'\[.*?\]', '', name).strip()
    return name

In [75]:
# 프로그램 이름 정규화 적용 - 회차 제거
vod_5['asset_nm'] = vod_5['asset_nm'].apply(normalize_program_name_remain)

contents_5['asset_nm'] = contents_5['asset_nm'].apply(normalize_program_name_remain)

C:\Users\USER\AppData\Local\Temp\ipykernel_7780\3456192095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_5['asset_nm'] = vod_5['asset_nm'].apply(normalize_program_name_remain)
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\3456192095.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_5['asset_nm'] = contents_5['asset_nm'].apply(normalize_program_name_remain)


In [76]:
vod_5['strt_date'] = vod_5['datetime'].dt.date
contents_5['strt_date'] = contents_5['datetime'].dt.date


vod_5['time'] = vod_5['datetime'].dt.time
contents_5['time'] = contents_5['datetime'].dt.time

C:\Users\USER\AppData\Local\Temp\ipykernel_7780\1923516231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_5['strt_date'] = vod_5['datetime'].dt.date
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\1923516231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_5['strt_date'] = contents_5['datetime'].dt.date
C:\Users\USER\AppData\Local\Temp\ipykernel_7780\1923516231.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [77]:
first_start_times = vod_5.groupby(['subsr', 'asset_nm']).agg({'use_tms': 'sum', 'strt_date' : 'min', 'datetime' : 'min'}).reset_index()
first_access_times = contents_5.groupby(['subsr', 'asset_nm']).agg({'strt_date' : 'min', 'datetime' : 'min'}).reset_index()
# print(first_start_times) #9852
# print(first_access_times) #26205

In [78]:
first_access_times

,subsr,asset_nm,strt_date,datetime
0,59879000,간신,2024-01-19,2024-01-19 05:26:23
1,59879000,남편은 이렇게 못해줘,2024-01-09,2024-01-09 20:28:44
2,59880000,도그데이즈,2024-03-09,2024-03-09 09:22:41
3,59882000,한글용사 아이야 시즌3 4회,2024-03-24,2024-03-24 08:32:25
4,59883000,마크맨,2024-02-15,2024-02-15 22:53:40
...,...,...,...,...
26200,67400000,아내 대신 채워준 야한 장모님,2024-03-31,2024-03-31 22:32:16
26201,67400000,유산으로 남겨준 새엄마,2024-03-31,2024-03-31 22:31:52
26202,67400000,조건섹스 : 라스트 원나잇,2024-03-31,2024-03-31 22:49:22
26203,67400000,편의점 털다 아랫도리 털린 유부녀,2024-03-31,2024-03-31 22:31:58


In [79]:
content_based_merged = pd.merge(first_access_times, first_start_times, on=['subsr', 'asset_nm'], how='left', suffixes=('_access', '_start'))

In [80]:
content_based_merged['time_difference'] = content_based_merged['datetime_start'] - content_based_merged['datetime_access']
content_based_merged #21393

,subsr,asset_nm,strt_date_access,datetime_access,use_tms,strt_date_start,datetime_start,time_difference
0,59879000,간신,2024-01-19,2024-01-19 05:26:23,NaN,NaN,NaT,NaT
1,59879000,남편은 이렇게 못해줘,2024-01-09,2024-01-09 20:28:44,940.0,2024-01-09,2024-01-09 20:28:46,0 days 00:00:02
2,59880000,도그데이즈,2024-03-09,2024-03-09 09:22:41,NaN,NaN,NaT,NaT
3,59882000,한글용사 아이야 시즌3 4회,2024-03-24,2024-03-24 08:32:25,NaN,NaN,NaT,NaT
4,59883000,마크맨,2024-02-15,2024-02-15 22:53:40,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...
26200,67400000,아내 대신 채워준 야한 장모님,2024-03-31,2024-03-31 22:32:16,NaN,NaN,NaT,NaT
26201,67400000,유산으로 남겨준 새엄마,2024-03-31,2024-03-31 22:31:52,NaN,NaN,NaT,NaT
26202,67400000,조건섹스 : 라스트 원나잇,2024-03-31,2024-03-31 22:49:22,NaN,NaN,NaT,NaT
26203,67400000,편의점 털다 아랫도리 털린 유부녀,2024-03-31,2024-03-31 22:31:58,NaN,NaN,NaT,NaT


In [81]:
#5초 미만 제거
content_based_merged = content_based_merged[(content_based_merged['time_difference'].isna()) | (content_based_merged['time_difference'] >= pd.Timedelta(seconds=5))]
content_based_merged ##24103

,subsr,asset_nm,strt_date_access,datetime_access,use_tms,strt_date_start,datetime_start,time_difference
0,59879000,간신,2024-01-19,2024-01-19 05:26:23,NaN,NaN,NaT,NaT
2,59880000,도그데이즈,2024-03-09,2024-03-09 09:22:41,NaN,NaN,NaT,NaT
3,59882000,한글용사 아이야 시즌3 4회,2024-03-24,2024-03-24 08:32:25,NaN,NaN,NaT,NaT
4,59883000,마크맨,2024-02-15,2024-02-15 22:53:40,NaN,NaN,NaT,NaT
5,59883000,서울의 봄,2024-02-15,2024-02-15 22:23:13,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...
26200,67400000,아내 대신 채워준 야한 장모님,2024-03-31,2024-03-31 22:32:16,NaN,NaN,NaT,NaT
26201,67400000,유산으로 남겨준 새엄마,2024-03-31,2024-03-31 22:31:52,NaN,NaN,NaT,NaT
26202,67400000,조건섹스 : 라스트 원나잇,2024-03-31,2024-03-31 22:49:22,NaN,NaN,NaT,NaT
26203,67400000,편의점 털다 아랫도리 털린 유부녀,2024-03-31,2024-03-31 22:31:58,NaN,NaN,NaT,NaT


In [82]:
#기본적으로 inner join - 왼쪽 데이터 프레임을 기준으로 병합하고 오른쪽프레임에 해당하는 행이 없는 경우 행을 삭제
#datetime_access가 없는 경우에 drop함
vod_content_merged_num = pd.merge(first_start_times, first_access_times, on=['subsr', 'asset_nm'], suffixes=('_start', '_access'))
vod_content_merged_num #3506

,subsr,asset_nm,use_tms,strt_date_start,datetime_start,strt_date_access,datetime_access
0,59879000,남편은 이렇게 못해줘,940,2024-01-09,2024-01-09 20:28:46,2024-01-09,2024-01-09 20:28:44
1,59883000,표적,1310,2024-02-15,2024-02-15 22:31:28,2024-02-15,2024-02-15 22:31:11
2,59899000,0시,160,2024-03-01,2024-03-01 21:27:35,2024-03-01,2024-03-01 21:27:32
3,59899000,겨울부인,119,2024-03-01,2024-03-01 21:39:13,2024-03-01,2024-03-01 21:39:10
4,59899000,경마장가는길,456,2024-03-01,2024-03-01 21:41:26,2024-03-01,2024-03-01 21:41:24
...,...,...,...,...,...,...,...
4856,67348000,놀면 뭐하니? 227회,627,2024-03-25,2024-03-25 12:44:28,2024-03-25,2024-03-25 12:54:55
4857,67356000,블라인드,3890,2024-03-18,2024-03-18 19:36:56,2024-03-18,2024-03-18 19:36:33
4858,67356000,색에 놀다,1966,2024-02-25,2024-02-25 19:24:02,2024-02-25,2024-02-25 19:23:45
4859,67356000,섹스 앤 머니,5271,2024-02-26,2024-02-26 08:39:23,2024-02-26,2024-02-26 08:39:10


In [83]:
vod_content_prior = vod_content_merged_num[vod_content_merged_num['strt_date_start'] > vod_content_merged_num['strt_date_access']]
vod_content_prior #740

vod_content_under_5 = vod_content_prior[vod_content_prior['use_tms'] < 300]
vod_content_under_5

,subsr,asset_nm,use_tms,strt_date_start,datetime_start,strt_date_access,datetime_access
29,59937000,색에 놀다,133,2024-02-21,2024-02-21 01:52:01,2024-02-03,2024-02-03 23:39:10
198,60320000,1947 보스톤,114,2024-03-15,2024-03-15 23:15:17,2024-02-25,2024-02-25 23:03:25
627,60495000,라이브 플래쉬,299,2024-03-05,2024-03-05 14:28:56,2024-01-08,2024-01-08 14:18:03
854,60784000,30대 숫처녀와 경험한 적 없는 섹스,122,2024-02-09,2024-02-09 20:39:54,2024-01-07,2024-01-07 22:44:11
855,60784000,99번의 정사,295,2024-01-07,2024-01-07 00:05:16,2024-01-06,2024-01-06 22:24:43
...,...,...,...,...,...,...,...
4329,66496000,쾌락의 밤,224,2024-01-20,2024-01-20 11:48:12,2024-01-09,2024-01-09 19:30:19
4457,66705000,귀수: 고스트 킬러,64,2024-03-20,2024-03-20 17:44:12,2024-03-16,2024-03-16 08:28:11
4486,66705000,본 투 플라이,213,2024-02-16,2024-02-16 23:32:17,2024-01-24,2024-01-24 20:16:16
4526,66705000,왜구 토벌대,148,2024-02-21,2024-02-21 16:34:07,2024-01-18,2024-01-18 19:51:53


In [84]:
print(len(vod_content_prior) / len(content_based_merged) * 100)
print(len(vod_content_prior) / len(first_start_times) * 100)

3.2693025764427666
7.998375964271213


In [85]:
print(len(vod_content_under_5)/len(vod_content_prior))

0.1319796954314721


###### 상세페이지 접속의 vod 시청 전환율)  VOD 전환 / 상세페이지 = 3.2693%
###### vod 시청 기록 중 상세페이지에서 넘어온 비율) 725 / 4066 = 7.9984%